# Implementation of Probabilistic Canonical Correlation Analysis

In [35]:
import numpy as np
inv = np.linalg.inv

## The function below implements EM Algorithm

In [36]:
def expectation_maximize(params):
  Psi = params['Psi']
  k = params['k']
  W = params['W']
  X = params['X']
  n = params['n']
  p1 = params['p1']
  p2 = params['p2']
  Sigma1 = params['Sigma1']
  Sigma2 = params['Sigma2']

  Psi_inv = inv(Psi)
  M = inv(np.eye(k) + W.T @ Psi_inv @ W)
  S = M @ W.T @ Psi_inv @ X
  A = n * M + S @ S.T

  W_new = X @ S.T @ inv(A)

  W1       = W[:p1]
  W1_new   = W_new[:p1]
  Psi1_inv = Psi_inv[:p1, :p1]
  Psi1_new = Sigma1 - Sigma1 @ Psi1_inv @ W1 @ M @ W1_new.T

  W2       = W[p1:]
  W2_new   = W_new[p1:]
  Psi2_inv = Psi_inv[p1:, p1:]
  Psi2_new = Sigma2 - Sigma2 @ Psi2_inv @ W2 @ M @ W2_new.T

  Psi_new = np.block([[Psi1_new, np.zeros((p1, p2))],
                      [np.zeros((p2, p1)), Psi2_new]])

  params['W'] = W_new
  params['Psi'] = Psi_new

## Implemtation of fit function below

In [37]:
def fit(params):
  Psi = params['Psi']
  iters = params['iters']
  np.linalg.cholesky(Psi)
  for _ in range(iters):
    print(_)
    expectation_maximize(params)
    Psi = params['Psi']
    np.linalg.cholesky(Psi)

## Below is the implementation of transform function

In [38]:
def transform(params):
  X1 = params['X1']
  X2 = params['X2']
  Psi = params['Psi']
  k = params['k']
  W = params['W']
  
  X = np.hstack([X1, X2]).T
  Psi_inv = inv(Psi)
  M = inv(np.eye(k) + W.T @ Psi_inv @ W)
  Z = M @ W.T @ Psi_inv @ X
  return Z.T

## The given function fits the trained data and transforms it to reduced dimensions

In [39]:
def fit_transform(params):
  fit(params)
  return transform(params)

## This function returns the projected X1 and X2 matrix

In [40]:
def get_projections(params, n_samples):
  Psi = params['Psi']
  k = params['k']
  W = params['W']
  X = params['X']
  p = params['p']
  p1 = params['p1']

  Psi_inv = inv(Psi)
  M = inv(np.eye(k) + W.T @ Psi_inv @ W)
  Z_post_mean = M @ W.T @ Psi_inv @ X

  X_mean = W @ Z_post_mean
  X = np.zeros((n_samples, p))

  for i in range(n_samples):
    X[i] = np.random.multivariate_normal(X_mean[:, i], Psi)

  return X[:, :p1], X[:, p1:]

## This function calls the fit_transform and get_projections function and returns the reduced Z matrix

In [41]:
def pcca(inmats, nvecs, iters = 500, nprojs = 100):
  # arguments
  # inmats : input matrices [X1,X2]
  # nvecs : number of reduced dimensions
  # iters : number of iterations for optimization
  # nprojs : number of projections to generate

  X1 = inmats[0]
  X2 = inmats[1]
  n = X1.shape[0]
  p1 = X1.shape[1]
  p2 = X2.shape[1]
  p = p1 + p2
  X = np.hstack([X1, X2]).T
  k = nvecs

  #covariances
  Sigma1 = np.cov(X1.T)
  Sigma2 = np.cov(X2.T)

  # Initialize Weights
  W1 = np.random.random((p1, k))
  W2 = np.random.random((p2, k))
  W = np.vstack([W1, W2])

  # Initialize Psi.
  prior_var1 = 1
  prior_var2 = 1
  Psi1 = prior_var1 * np.eye(p1)
  Psi2 = prior_var2 * np.eye(p2)
  Psi = np.block([[Psi1, np.zeros((p1, p2))],
                  [np.zeros((p2, p1)), Psi2]])
  
  params = {}
  params['k'] = k
  params['iters'] = iters
  params['X1'] = X1
  params['X2'] = X2
  params['n'] = n
  params['p1'] = p1
  params['p2'] = p2
  params['p'] = p
  params['X'] = X
  params['Sigma1'] = Sigma1
  params['Sigma2'] = Sigma2
  params['W'] = W
  params['Psi'] = Psi

  Z = fit_transform(params)
  X1, X2 = get_projections(params, nprojs)
  return Z,X1,X2

## Below are two examples of PCCA API implemented in the above code

In [46]:
# Example1
N = 10 # number of instances
NVECS = 2 # number of reduced dimensions

# Generate random data
X = np.random.rand(N, 4)
Y = np.random.rand(N, 5)

# Center X to it's mean
X_mean = X.mean(axis = 0)
X = X - X_mean
# Center Y to it's mean
Y_mean = Y.mean(axis = 0)
Y = Y - Y_mean

Z, X1, X2 = pcca([X, Y], nvecs = NVECS, nprojs = N)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [47]:
print(Z)

[[-0.92685876 -1.24807945]
 [-0.08401839 -1.00570488]
 [ 1.62996732 -0.30235556]
 [-0.59345041  0.6316667 ]
 [ 0.47387167  0.08871249]
 [ 0.23720424 -0.97758628]
 [ 0.74136739  0.8026159 ]
 [-0.74737633  0.80332649]
 [-0.10402118  0.73764799]
 [-0.62668555  0.46975659]]


In [54]:
# Example2
N = 10 # number of instances
NVECS = 3 # number of reduced dimensions

# Generate random data
X = np.random.rand(N, 6)
Y = np.random.rand(N, 8)

# Center X to it's mean
X_mean = X.mean(axis = 0)
X = X - X_mean
# Center Y to it's mean
Y_mean = Y.mean(axis = 0)
Y = Y - Y_mean

Z, X1, X2 = pcca([X, Y], nvecs = NVECS, nprojs = N)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [55]:
print(Z)

[[ 0.03758889 -1.02736095  0.52083724]
 [-1.02062077  0.62835255 -1.38952221]
 [ 0.36994658 -1.06419378 -0.05873589]
 [ 1.47276287  0.20190599 -0.94186114]
 [-0.22459376 -0.00332402 -0.20935278]
 [ 0.33850169 -0.1029961   0.36483614]
 [-0.59834667 -0.65666066  0.98148924]
 [-1.20945419 -0.11301478 -0.4685088 ]
 [ 0.9024395   0.43949108 -0.09977397]
 [-0.06822414  1.69780066  1.30059217]]
